## Standing Charge Price Cap

In [2]:
#https://www.ofgem.gov.uk/energy-price-cap
ele_pricecap_day_GBP = 0.6099
gas_pricecap_day_GBP = 0.3166
StandingCharge_ele_yr_GBP = ele_pricecap_day_GBP * 365
StandingCharge_gas_yr_GBP = gas_pricecap_day_GBP * 365
kWh_price_ele_GBP = 0.2450 #including VAT
kWh_price_gas_GBP = 0.0604 #including VAT

## Levy yearly amounts

In [18]:
#https://eciu.net/insights/2024/are-green-levies-going-up-in-april-2024

PriceCapBill_ele_GBP = 839
PriceCapBill_gas_GBP = 771
PriceCapBill_ele_VATincl_GBP = 881
PriceCapBill_gas_VATincl_GBP = 809

#Renewable Obligation
RO_ele_GBP = 86
RO_gas_GBP = 0
#Feed-in-Tariff
FIT_ele_GBP = 21
FIT_gas_GBP = 0
#Energy Company Obligation
ECO_ele_GBP = 23
ECO_gas_GBP = 34
#Warm Homes Discount
WHD_ele_GBP = 11
WHD_gas_GBP = 11
#AAHEDC (GB Average)
AAHEDC_ele_GBP = 1
AAHEDC_gas_GBP = 0
#Green Gas Levy
GGL_ele_GBP = 0
GGL_gas_GBP = 0.4
#VAT on levies
VAT_ele_GBP = 7
VAT_gas_GBP = 2

yearly_levy_ele_GBP = RO_ele_GBP + FIT_ele_GBP + ECO_ele_GBP + WHD_ele_GBP + AAHEDC_ele_GBP + GGL_ele_GBP
yearly_levy_gas_GBP = RO_gas_GBP + FIT_gas_GBP + ECO_gas_GBP + WHD_gas_GBP + AAHEDC_gas_GBP + GGL_gas_GBP
yearly_levy_dual_GBP = yearly_levy_ele_GBP + yearly_levy_gas_GBP

## Levy unit rates
#### Assuming levies except GGL are added onto the unit rate, not standing charge

In [17]:
#Calculate yearly bill without SC
YearlyBill_ele_exclSC_VATincl_GBP = PriceCapBill_ele_VATincl_GBP - StandingCharge_ele_yr_GBP
YearlyBill_gas_exclSC_VATincl_GBP = PriceCapBill_gas_VATincl_GBP - StandingCharge_gas_yr_GBP

#Calculate unit price per kWh used in https://eciu.net/insights/2024/are-green-levies-going-up-in-april-2024
ele_levies_per_kWh = YearlyBill_ele_exclSC_VATincl_GBP/kWh_price_ele_GBP
gas_levies_per_kWh = YearlyBill_gas_exclSC_VATincl_GBP/kWh_price_gas_GBP

#Find levy as unit rate excluding VAT
ele_levy_unit_rate_GBP_kWh = yearly_levy_ele_GBP/ele_levies_per_kWh
gas_levy_unit_rate_GBP_kWh = yearly_levy_gas_GBP/gas_levies_per_kWh
#Find levy as unit rate including VAT
ele_levy_unit_rate_GBP_kWh_VATincl = ele_levy_unit_rate_GBP_kWh + VAT_ele_GBP/ele_levies_per_kWh
gas_levy_unit_rate_GBP_kWh_VATincl = gas_levy_unit_rate_GBP_kWh + VAT_gas_GBP/gas_levies_per_kWh


# EST Factors

In [36]:
EST_ele_unit_price_GBP = 0.22360
EST_E7_off_unit_price_GBP = 0.13047
EST_E7_on_unit_price_GBP = 0.26687

EST_gas_unit_price_GBP = 0.05480
EST_ele_standing_charge_GBP_yr = 219.44
EST_gas_standing_charge_GBP_yr = 114.65
EST_Gas_Boiler_Eff = 0.778

# Baseline scenario

In [103]:
TypicalHomeUse_ele = 2700
LowHomeUse_ele = 1800
HighHomeUse_ele = 4100

TypicalHomeUse_gas = 11500
LowHomeUse_gas = 7500
HighHomeUse_gas = 17000

TypicalHeatDemand = TypicalHomeUse_gas*EST_Gas_Boiler_Eff
LowHeatDemand     = LowHomeUse_gas*EST_Gas_Boiler_Eff
HighHeatDemand    = HighHomeUse_gas*EST_Gas_Boiler_Eff

def getBaselineBill(HomeUse_ele,HomeUse_gas,type="gas",perc_offpeak=0.9,SPF=2):
    HeatDemand     = HomeUse_gas*EST_Gas_Boiler_Eff
    if type=="gas":
        return HomeUse_ele*EST_ele_unit_price_GBP + EST_ele_standing_charge_GBP_yr + HomeUse_gas*EST_gas_unit_price_GBP + EST_gas_standing_charge_GBP_yr
    if type=="E7":
        return (HomeUse_ele + (1-perc_offpeak)*HeatDemand)*EST_E7_on_unit_price_GBP + perc_offpeak*HeatDemand*EST_E7_off_unit_price_GBP + EST_ele_standing_charge_GBP_yr
    if type=="heatpump":
        return (HomeUse_ele + HeatDemand/SPF)*EST_ele_unit_price_GBP + EST_ele_standing_charge_GBP_yr
    if type=="heatpump_gascook":
        return (HomeUse_ele + HeatDemand/SPF)*EST_ele_unit_price_GBP + EST_ele_standing_charge_GBP_yr + EST_gas_standing_charge_GBP_yr
    else:
        print("wrong type")

def BaselineHPswitch(HomeUse_ele,HomeUse_gas,SPF,beforetype="gas",gasStandingCharge=False,perc_offpeak=0.9):
    if beforetype=="gas":
        gasboiler = getBaselineBill(HomeUse_ele,HomeUse_gas,type=beforetype)
        if gasStandingCharge==True:
            heatpump = getBaselineBill(HomeUse_ele,HomeUse_gas,type="heatpump_gascook",SPF=SPF)
            return gasboiler - heatpump
        else:
            heatpump = getBaselineBill(HomeUse_ele,HomeUse_gas,type="heatpump",SPF=SPF)
            return gasboiler - heatpump
        
    if beforetype=="E7":
        storageheater = getBaselineBill(HomeUse_ele,HomeUse_gas,type=beforetype,perc_offpeak=perc_offpeak)
        if gasStandingCharge==True:
            heatpump = getBaselineBill(HomeUse_ele,HomeUse_gas,type="heatpump_gascook",SPF=SPF)
            return storageheater - heatpump
        else:
            heatpump = getBaselineBill(HomeUse_ele,HomeUse_gas,type="heatpump",SPF=SPF)
            return storageheater - heatpump
    

BaselineLowBill_gashome     = getBaselineBill(LowHomeUse_ele,LowHomeUse_gas,type="gas")
BaselineTypicalBill_gashome = getBaselineBill(TypicalHomeUse_ele,TypicalHomeUse_gas,type="gas")
BaselineHighBill_gashome    = getBaselineBill(HighHomeUse_ele,HighHomeUse_gas,type="gas")

#E7 no gas SC, X percent of heating is off peak
off_peak_heating = 0.9
BaselineLowBill_E7     = getBaselineBill(LowHomeUse_ele,LowHomeUse_gas,type="E7",perc_offpeak=off_peak_heating)
BaselineTypicalBill_E7 = getBaselineBill(TypicalHomeUse_ele,TypicalHomeUse_gas,type="E7",perc_offpeak=off_peak_heating)
BaselineHighBill_E7    = getBaselineBill(HighHomeUse_ele,HighHomeUse_gas,type="E7",perc_offpeak=off_peak_heating)


In [105]:
BaselineHPswitch(LowHomeUse_ele,LowHomeUse_gas,beforetype="gas",gasStandingCharge=False,SPF=2)

-126.70299999999975

# 1. Move all levies completely off bills and onto general taxation

In [98]:
NoLevy_ele_unit_price = EST_ele_unit_price_GBP-ele_levy_unit_rate_GBP_kWh_VATincl
NoLevy_gas_unit_price = EST_gas_unit_price_GBP-gas_levy_unit_rate_GBP_kWh_VATincl

NoLevy_E7_peak_price = EST_E7_on_unit_price_GBP - ele_levy_unit_rate_GBP_kWh_VATincl
NoLevy_E7_off_price = EST_E7_off_unit_price_GBP - ele_levy_unit_rate_GBP_kWh_VATincl

def getNoLevyBill(HomeUse_ele,HomeUse_gas,type="gas",perc_offpeak=0.9,SPF=2):
    HeatDemand     = HomeUse_gas*EST_Gas_Boiler_Eff
    if type=="gas":
        return HomeUse_ele*NoLevy_ele_unit_price + EST_ele_standing_charge_GBP_yr + HomeUse_gas*NoLevy_gas_unit_price + EST_gas_standing_charge_GBP_yr - 1.05*GGL_gas_GBP #1.05 to add VAT onto the GGL
    if type=="E7":
        return (HomeUse_ele + (1-perc_offpeak)*HeatDemand)*NoLevy_E7_peak_price + perc_offpeak*HeatDemand*NoLevy_E7_off_price + EST_ele_standing_charge_GBP_yr
    if type=="heatpump":
        return (HomeUse_ele + HeatDemand/SPF)*NoLevy_ele_unit_price + EST_ele_standing_charge_GBP_yr
    if type=="heatpump_gascook":
        return (HomeUse_ele + HeatDemand/SPF)*NoLevy_ele_unit_price + EST_ele_standing_charge_GBP_yr + EST_gas_standing_charge_GBP_yr - 1.05*GGL_gas_GBP #1.05 to add VAT onto the GGL
    else:
        print("wrong type")

def scenario1saving(HomeUse_ele,HomeUse_gas,type="gas",perc_offpeak=0.9,SPF=2):
    saving = getBaselineBill(HomeUse_ele,HomeUse_gas,type=type,perc_offpeak=perc_offpeak,SPF=SPF) - getNoLevyBill(HomeUse_ele,HomeUse_gas,type=type,perc_offpeak=perc_offpeak,SPF=SPF)
    return saving
def scenario1HPswitch(HomeUse_ele,HomeUse_gas,SPF,beforetype="gas",gasStandingCharge=False,perc_offpeak=0.9):
    if beforetype=="gas":
        gasboiler = getNoLevyBill(HomeUse_ele,HomeUse_gas,type=beforetype)
        if gasStandingCharge==True:
            heatpump = getNoLevyBill(HomeUse_ele,HomeUse_gas,type="heatpump_gascook",SPF=SPF)
            return gasboiler - heatpump
        else:
            heatpump = getNoLevyBill(HomeUse_ele,HomeUse_gas,type="heatpump",SPF=SPF)
            return gasboiler - heatpump
        
    if beforetype=="E7":
        storageheater = getNoLevyBill(HomeUse_ele,HomeUse_gas,type=beforetype,perc_offpeak=perc_offpeak)
        if gasStandingCharge==True:
            heatpump = getNoLevyBill(HomeUse_ele,HomeUse_gas,type="heatpump_gascook",SPF=SPF)
            return storageheater - heatpump
        else:
            heatpump = getNoLevyBill(HomeUse_ele,HomeUse_gas,type="heatpump",SPF=SPF)
            return storageheater - heatpump
    

## Now model heat pump depending on SPF

In [102]:
import numpy as np

test = np.linspace(0.1,5,100)
scenario1HPswitch(LowHomeUse_ele,LowHomeUse_gas,beforetype="E7",gasStandingCharge=False,SPF=test)

array([-9.21653731e+03, -5.96804363e+03, -4.33555188e+03, -3.35340237e+03,
       -2.69752626e+03, -2.22849857e+03, -1.87642944e+03, -1.60242089e+03,
       -1.38310244e+03, -1.20358622e+03, -1.05393858e+03, -9.27277562e+02,
       -8.18684640e+02, -7.24551101e+02, -6.42169265e+02, -5.69467789e+02,
       -5.04835106e+02, -4.46998564e+02, -3.94939780e+02, -3.47834162e+02,
       -3.05006904e+02, -2.65900455e+02, -2.30050107e+02, -1.97065447e+02,
       -1.66616079e+02, -1.38420516e+02, -1.12237456e+02, -8.78588533e+01,
       -6.51043863e+01, -4.38169835e+01, -2.38591955e+01, -5.11022284e+00,
        1.25365307e+01,  2.91754859e+01,  4.48905785e+01,  5.97566749e+01,
        7.38407652e+01,  8.72029721e+01,  9.98974095e+01,  1.11972914e+02,
        1.23473674e+02,  1.34439768e+02,  1.44907635e+02,  1.54910474e+02,
        1.64478600e+02,  1.73639748e+02,  1.82419345e+02,  1.90840739e+02,
        1.98925415e+02,  2.06693172e+02,  2.14162286e+02,  2.21349655e+02,
        2.28270926e+02,  